In [2]:
from transformers import TapasTokenizer, TapasForQuestionAnswering
import pandas as pd

In [3]:
tokenizer = TapasTokenizer.from_pretrained("google/tapas-large-finetuned-wtq")
model = TapasForQuestionAnswering.from_pretrained("google/tapas-large-finetuned-wtq")

tokenizer_config.json:   0%|          | 0.00/490 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/262k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

In [13]:
def apply_model(table, queries):
    #inputs = tokenizer(table=table, queries=queries, padding="max_length", return_tensors="pt")
    inputs = tokenizer(table=table, queries=queries, padding="max_length", truncation=True, return_tensors="pt")

    outputs = model(**inputs)

    predicted_answer_coordinates, predicted_aggregation_indices = tokenizer.convert_logits_to_predictions(
        inputs, outputs.logits.detach(), outputs.logits_aggregation.detach()
    )

    id2aggregation = {0: "NONE", 1: "SUM", 2: "AVERAGE", 3: "COUNT"}
    aggregation_predictions_string = [id2aggregation[x] for x in predicted_aggregation_indices]

    answers = []

    for coordinates in predicted_answer_coordinates:
        if len(coordinates) == 1:
            # only a single cell:
            answers.append(table.iat[coordinates[0]])
        else:
            # multiple cells
            cell_values = []
            for coordinate in coordinates:
                cell_values.append(table.iat[coordinate])
            answers.append(", ".join(cell_values))

    display(table)
    #print("")
    result = ""

    for query, answer, predicted_agg in zip(queries, answers, aggregation_predictions_string):
        question = '===> {}\n'.format(query)
        if predicted_agg == "NONE":
            #print('{}\n'.format(answer))
            answer_formatted = answer + '\n'
        else:
            answer_formatted = '{} > {}\n\n'.format(predicted_agg, answer)
        result += question + answer_formatted

    return result


In [4]:
# df = pd.read_('https://raw.githubusercontent.com/kemperd/tables/main/Countries%20by%20GDP.csv', sep=';')
# df = df.reset_index(drop=True)

# # Pandas guesses the numerical columns to be numeric, but TAPAS only like strings
# df['GDP'] = df['GDP'].astype(str)
# df['Year'] = df['Year'].astype(str)

In [6]:
import requests
import pandas as pd

# URL containing the JSON data
url = "https://cea.nic.in/api/psp_energy.php"

# Make a request to the URL
response = requests.get(url)

# Assuming the response is successful and in JSON format
data = response.json()

# Convert the JSON data into a pandas DataFrame
# Since the JSON is nested, you might need to adjust the path to the data you need
df = pd.json_normalize(data, record_path=['2019-2020'])

# Now you can use df for your analysis
print(df.head())


   ID     Month                     State energy_requirement  \
0  44  Jan-2020                Chandigarh             137.98   
1  45  Jan-2020                     Delhi            2290.02   
2  46  Jan-2020                   Haryana            3862.87   
3  47  Jan-2020          Himachal Pradesh             931.24   
4  48  Jan-2020   \nUTs of J&K and Ladakh               1934   

  energy_availability         fy  
0              137.98  2019-2020  
1             2290.02  2019-2020  
2             3862.87  2019-2020  
3              930.64  2019-2020  
4             1699.59  2019-2020  


In [10]:
df['ID']

0        44
1        45
2        46
3        47
4        48
       ... 
254    1650
255    1651
256    1652
257    1653
258    1654
Name: ID, Length: 259, dtype: object

In [11]:
df.columns

Index(['ID', 'Month', 'State', 'energy_requirement', 'energy_availability',
       'fy'],
      dtype='object')

In [12]:
df['ID'] = df['ID'].astype(str)
df['Month'] = df['Month'].astype(str)
df['State'] = df['State'].astype(str)
df['energy_requirement'] = df['energy_requirement'].astype(str)
df['energy_availability'] = df['energy_availability'].astype(str)
df['fy']=df['fy'].astype(str)

In [14]:
# queries_gdp = [
#     "What is the GDP of the United States in 2022?",
#     "What is the total GDP of Europe in 2022?",
#     "What is the average GDP of all countries in 2022?",
#     "What is the GDP of Italy in 2021?",
#     "How many distinct countries are there in Asia?",
#     "Total European GDP in 2021?",
#     "What countries have a GDP higher than 10 million in 2022?",
#     "How much does the total GDP in 2022 differ from the total GDP in 2021?",
#     "What is the percentual increase of the US 2022 GDP vs 2021?"
# ]

# print(apply_model(df, queries_gdp))

/opt/conda/lib/python3.10/site-packages/transformers/models/tapas/tokenization_tapas.py:2762: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/conda/lib/python3.10/site-packages/transformers/models/tapas/tokenization_tapas.py:1561: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


KeyboardInterrupt: 

In [15]:
queries_gdp = [
    "What was the energy requirement for Delhi in Jan-2020?",
"How much energy was available in Haryana in Jan-2020?",
"Compare the energy availability with the requirement for Himachal Pradesh in Jan-2020.",
"List the states with an energy requirement greater than 2000 in Jan-2020.",
"What is the total energy requirement for all states combined in Jan-2020?",
"Which state had the highest energy availability in Jan-2020?",
"Calculate the difference in energy requirement and availability for Uttar Pradesh in Jan-2020.",
"What is the average energy requirement across all states in Jan-2020?",
"Identify states where energy availability met the requirement in Jan-2020.",
"How does the energy requirement for Punjab in Jan-2020 compare to that of Rajasthan?"
]

print(apply_model(df, queries_gdp))

,ID,Month,State,energy_requirement,energy_availability,fy
0,44,Jan-2020,Chandigarh,137.98,137.98,2019-2020
1,45,Jan-2020,Delhi,2290.02,2290.02,2019-2020
2,46,Jan-2020,Haryana,3862.87,3862.87,2019-2020
3,47,Jan-2020,Himachal Pradesh,931.24,930.64,2019-2020
4,48,Jan-2020,\nUTs of J&K and Ladakh,1934,1699.59,2019-2020
...,...,...,...,...,...,...
254,1650,Mar-2020,Mizoram,59.20070788048,58.96839839999999,2019-2020
255,1651,Mar-2020,Nagaland,64.42117146149504,64.11040458286989,2019-2020
256,1652,Mar-2020,Tripura,110.4698284,110.4698284,2019-2020
257,1653,Mar-2020,North-Eastern Region,1223.653469172747,1196.706720240298,2019-2020


===> What was the energy requirement for Delhi in Jan-2020?
SUM > 2290.02

===> How much energy was available in Haryana in Jan-2020?
SUM > 3862.87

===> Compare the energy availability with the requirement for Himachal Pradesh in Jan-2020.
AVERAGE > 931.24

===> List the states with an energy requirement greater than 2000 in Jan-2020.
 Delhi,  Maharashtra,  Western Region,  Andhra Pradesh,  Telangana,  Karnataka,  Kerala,  Southern Region,  Bihar,  Odisha,  West Bengal,  Eastern Region
===> What is the total energy requirement for all states combined in Jan-2020?
SUM > 137.98, 2290.02, 3862.87, 930.64, 1699.59, 3396.62, 7366.630000000001, 8725.869999999999, 1243.45, 29653.67, 2372.367499886316, 9440.014150077439, 7423.429520468, 13178.148249, 219.054645, 567.5795220000001, 413.7986939999999, 33614.39228043176, 5363.846723991, 6346.742839031, 7013.517095245116, 2239.295119748, 8708.320210381, 211.598856531, 3.85, 29883.32084492712, 2447.672791506999, 1933.293823883, 777.7239355490003, 

In [2]:
!pip install gemini_pro_client

ERROR: Could not find a version that satisfies the requirement gemini_pro_client (from versions: none)
ERROR: No matching distribution found for gemini_pro_client


In [1]:
import pandas as pd
import json
import os
from dotenv import load_dotenv
# Assuming Gemini Pro API client library is available
import gemini_pro_client as gemini

ModuleNotFoundError: No module named 'gemini_pro_client'